In [2]:
# Import modules

import pandas as pd
import numpy as np
import re
import datetime as dt #for working with times objects

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# For working with regression
import sklearn
import warnings
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

# For evaluating regression algorithms
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# For working with polynomial regression
from sklearn.preprocessing import PolynomialFeatures

